In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pytelegrambotapi

In [ ]:
!pip install fasttext
!pip install pymorphy2

In [ ]:
!pip install deeppavlov

In [ ]:
!pip install tensorflow==1.15

In [ ]:
!pip install transformers

In [ ]:
!pip install "scikit_learn==0.22.2.post1"

In [ ]:
import telebot

from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

import pandas as pd
import numpy as np
import seaborn as sns

import fasttext
import fasttext.util

from spacy.lang.ru import Russian
import nltk
from nltk.corpus import stopwords
from string import punctuation

from joblib import dump, load

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

nltk.download("stopwords")

device = torch.device("cuda")

In [ ]:
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

In [ ]:
log_reg_elmo = load('drive/MyDrive/log_reg_elmo.joblib')

In [7]:
russian_stopwords = stopwords.words("russian")
nlp = Russian()

def preprocess_text_elmo(text):
  tokens = text.lower()
  sentence = nlp(text.lower())
  tokens = [token.text for token in sentence]
  tokens = [token for token in tokens if token not in russian_stopwords\
            and token != " " \
            and token.strip() not in punctuation]
  
  
  return tokens


In [8]:
d = {
    0:'Курсы валют',
    1:'Условия - Комиссии',
    2:'Ограничения - Лимиты',
    3:'Запрос на оформление потребительского кредита',
    4:'Уточнить статус заявки по потребительскому кредиту: сумма, ставка, срок, дальнейшие действия',
    5:'Запрос основной информации по страховке по потребительскому кредиту',
    6:'Заказ кредитной карты',
    7:'Заказ дебетовой карты',
    8:'Заявка на ипотечное кредитование',
    9:'Заявка на рефинансирование ипотечного кредита',
    10:'Уточнить статус заявки по ипотечному кредиту: сумма, ставка, срок, дальнейшие действия',
    11:'Запрос основной информации по страховке по ипотечному кредиту'
}

In [11]:
def classify(text):

  elmo_emb = elmo([preprocess_text_elmo(text)])
  elmo_emb = elmo_emb.reshape((elmo_emb.shape[0], elmo_emb.shape[1]))

  log_reg_elmo_res = log_reg_elmo.predict(elmo_emb)


  return d[log_reg_elmo_res[0]] 

In [12]:
bot = telebot.TeleBot('1690424261:AAETkJ6T61-nXERmtnElWqpJonOFmpKemWw')

@bot.message_handler(commands=['start'])
def start_message(message):
  bot.send_message(message.chat.id, 'Пиши мне, а я буду отвечать')

@bot.message_handler(content_types=['text'])
def send_text(message):
  text = message.text
  bot.send_message(message.chat.id, classify(text))



bot.polling()